# Basic Image Stitching
This notebook contains the code that implements a simple stitching mechanism.
The idea is to find the corresponding points between the images and then after having defined an order between the images, stitch them one after the other. Thus, the first image is stitched with the second, the third is stitched with the result of the previous stitching, and so on.

## Importing libraries

In [1]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ipynb.fs.defs.Utils as Utils
import ipynb.fs.defs.ImageStitcher as ImageStitcher
from matplotlib.pyplot import figure

## Functions definition

In [ ]:
# This function computes a set of homographies following the structure of the Spanning Tree.
# The goal is to find an homography for each image that projects it in the reference frame of the "idx_ref" image.
def get_homographies_from_neighbors(graph,   # Graph (Maximum ST) used to define the ordering of the transformations to be applied to each image
                                    idx_ref, # Index of the reference image
                                    Z,       # Matrix containing the homographies between the images
                                   ):
    
    # Start from the reference image
    # For the first node the father is the node itself
    root_idx = idx_ref
    father_idx = idx_ref
    
    # Precompute the set of homographies, initialized to the identity
    H = [np.eye(3)]*len(graph.nodes)
    
    # Recursively updates the homographies in order to project the children of a node in the father reference frame.
    compute_homographies_from_pair(graph = graph,
                                   H = H, 
                                   root_idx = root_idx,
                                   father_idx = father_idx, 
                                   Z = Z)
    
    return H

In [ ]:
# Updates the homographies for all the children of the "root_idx" node.
# The homography that brings a child node "n" in the "idx_ref" can be computed by first projecting "n" in the father's frame ("root_idx"), and then using the father's projection stored in H.
def compute_homographies_from_pair(graph,       # Graph used to define the order of images to be stitched
                                    H,          # Set of homographies
                                    root_idx,   # Index of the current node
                                    father_idx, # Index of the father of the current node "root_idx"
                                    Z,          # Matrix containing the homographies between the nodes
                                  ):
    
    # Find the neighbors of the current node in the graph
    # Note: we know that the graph is a tree, but there is not a data-structure for them in this library, 
    # so we must remember that also the father of a node is returned when retrieving its neighbors.
    nb_it = graph.neighbors(root_idx) 
    
    # For each neighbor...
    for n in nb_it:
        #...that is a child, and not the father 
        if n != father_idx:
            
            # Define the homography that bring a child "n" in the global reference frame as:
            # - first project "n" in the father's frame ("root_idx"),
            # - then use the father's projection stored in H[root_idx].
            H[n] = H[root_idx] @ Z[(root_idx)*3:(root_idx+1)*3,(n)*3:(n+1)*3]
            
            #Call again the procedure to compute homographies of the descendants in the graph
            child = compute_homographies_from_pair(graph = graph,
                                                   H = H, 
                                                   root_idx = n,
                                                   father_idx = root_idx, 
                                                   Z = Z
                                                  )


In [ ]:
# This function can be used to manage the whole procedure of BasicStitching
# Returns: Set of normalized homographies (H), stitched image (stitch)
def basic_stitching(dataset_name,    # Name of the dataset to be used
                    imgs,            # Images to be stitched
                    T_norm,          # Normalization matrix
                    Z,               # Matrix containing the homographies between images
                    adj_matrix,      # Adjacency matrix of the graph built according to the matches between the images
                    weight_matrix,   # Weight matrix of the graph built according to the matches between the images
                    idx_ref = 0,     # Index of the reference image
                    verbose = True,  # If True prints additional graphs and images
                    beautify = True, # If True improves the quality of the stitched image
                    save_output = True, # If True saves the output of the procedure
                    stitching_dir = "stitched", # Name of the father directory
                    basic_stitching_dir = "basic_stitching", # Name of the directory where to store the final results
                    warp_shape = [10000,10000] # Shape of the stitched image
                   ):
    
    # Directory where to store the output
    output_dir = os.path.join(os.path.join(stitching_dir,dataset_name),basic_stitching_dir)
    
    # Check whether the directories already exist and create them if not
    if save_output:
        if os.path.isdir(output_dir):   
            shutil.rmtree(output_dir)
        if not os.path.isdir(output_dir):   
            os.makedirs(output_dir)
    
    # Build the graph starting from the images
    graph = Utils.build_graph(imgs, 
                              adj_matrix, 
                              weight_matrix)
    
    # Compute the maximum ST of the obtained graph and print it
    spanning_tree = nx.maximum_spanning_tree(graph)
    
    # Compute homographies
    H = get_homographies_from_neighbors(spanning_tree, idx_ref, Z)
    
    # Stitch images together
    stitcher = ImageStitcher.ImageStitcher(imgs)
    
    # Stitch all the images
    _,stitch = stitcher.stitch_images(H, idx_ref, T_norm , beautify=beautify, size = warp_shape) 
    
    # If required save the output
    if save_output:
        cv.imwrite(os.path.join(output_dir,"stitched.jpg"), cv.cvtColor(stitch,cv.COLOR_RGB2BGR))
    
  
    # If required plot the stitched image and intermediate results
    if verbose:
        Utils.advanced_print_graph(graph, 
                                  save_output, 
                                  output_dir,
                                  "graph")
        
        Utils.print_graph(spanning_tree, save_output, output_dir, "spanning-tree")
        figure(figsize=(40, 40), dpi=80)
        plt.imshow(stitch,),plt.show() 
    
    return H, stitch